In [1]:
import tqdm
import pandas as pd
import os
import sys

/home/ubuntu/miniconda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parent_dir)

from predict.prediction_models import (
    DensePredictor,
    ConvPredictor,
)

In [3]:
dense_predictor = DensePredictor(with_features=True)
conv_predictor = ConvPredictor(with_features=True)

In [4]:
dense_wo_features_predictor = DensePredictor(with_features=False)
conv_wo_features_predictor = ConvPredictor(with_features=False)

In [5]:
gpu_name = 'V100'

In [ ]:
# Initialize an empty dictionary to store the features of each convolutional layer
vgg16_conv_layers = {}
vgg16_dense_layers = {}
batch_size = 64
dataset_size = 1048

# Helper function to add layer details to the dictionary
def add_conv_layer_vgg(layer_name, matrix_size, kernel_size, channels_in, channels_out, strides, padding, activation, optimizer):
    vgg16_conv_layers[layer_name] = {
        'batchsize': batch_size,
        'matsize': matrix_size,
        'kernelsize': kernel_size,
        'channels_in': channels_in,
        'channels_out': channels_out,
        'strides': strides,
        'padding': padding,
        'activation_fct': activation,
        'optimizer': optimizer
    }
    
def add_dense_layer_vgg(layer_name, input_size, output_size, activation, optimizer):
    vgg16_dense_layers[layer_name] = {
        'batchsize': batch_size,
        'dim_input': input_size,
        'dim_output': output_size,
        'activation_fct': activation,
        'optimizer': optimizer
    }

# Block 1: 2 Conv layers + Max Pooling
add_conv_layer_vgg("block1_conv1", 224, 3, 3, 64, 1, 1, "ReLU", "Adam")
add_conv_layer_vgg("block1_conv2", 224, 3, 64, 64, 1, 1, "ReLU", "Adam")
# Max Pooling (2x2, stride 2) => reduces to 112x112

# Block 2: 2 Conv layers + Max Pooling
add_conv_layer_vgg("block2_conv1", 112, 3, 64, 128, 1, 1, "ReLU", "Adam")
add_conv_layer_vgg("block2_conv2", 112, 3, 128, 128, 1, 1, "ReLU", "Adam")
# Max Pooling (2x2, stride 2) => reduces to 56x56

# Block 3: 3 Conv layers + Max Pooling
add_conv_layer_vgg("block3_conv1", 56, 3, 128, 256, 1, 1, "ReLU", "Adam")
add_conv_layer_vgg("block3_conv2", 56, 3, 256, 256, 1, 1, "ReLU", "Adam")
add_conv_layer_vgg("block3_conv3", 56, 3, 256, 256, 1, 1, "ReLU", "Adam")
# Max Pooling (2x2, stride 2) => reduces to 28x28

# Block 4: 3 Conv layers + Max Pooling
add_conv_layer_vgg("block4_conv1", 28, 3, 256, 512, 1, 1, "ReLU", "Adam")
add_conv_layer_vgg("block4_conv2", 28, 3, 512, 512, 1, 1, "ReLU", "Adam")
add_conv_layer_vgg("block4_conv3", 28, 3, 512, 512, 1, 1, "ReLU", "Adam")
# Max Pooling (2x2, stride 2) => reduces to 14x14

# Block 5: 3 Conv layers + Max Pooling
add_conv_layer_vgg("block5_conv1", 14, 3, 512, 512, 1, 1, "ReLU", "Adam")
add_conv_layer_vgg("block5_conv2", 14, 3, 512, 512, 1, 1, "ReLU", "Adam")
add_conv_layer_vgg("block5_conv3", 14, 3, 512, 512, 1, 1, "ReLU", "Adam")
# Max Pooling (2x2, stride 2) => reduces to 7x7

# Print out the dictionary to verify
for layer_name, features in vgg16_conv_layers.items():
    print(f"{layer_name}: {features}")
    
    
# add dense layers
add_dense_layer_vgg("fc1", 25088, 4096, "ReLU", "Adam")
add_dense_layer_vgg("fc2", 4096, 4096, "ReLU", "Adam")
add_dense_layer_vgg("fc3", 4096, 10, "Softmax", "Adam")

block1_conv1: {'batchsize': 64, 'matsize': 224, 'kernelsize': 3, 'channels_in': 3, 'channels_out': 64, 'strides': 1, 'padding': 1, 'activation_fct': 'ReLU', 'optimizer': 'Adam'}
block1_conv2: {'batchsize': 64, 'matsize': 224, 'kernelsize': 3, 'channels_in': 64, 'channels_out': 64, 'strides': 1, 'padding': 1, 'activation_fct': 'ReLU', 'optimizer': 'Adam'}
block2_conv1: {'batchsize': 64, 'matsize': 112, 'kernelsize': 3, 'channels_in': 64, 'channels_out': 128, 'strides': 1, 'padding': 1, 'activation_fct': 'ReLU', 'optimizer': 'Adam'}
block2_conv2: {'batchsize': 64, 'matsize': 112, 'kernelsize': 3, 'channels_in': 128, 'channels_out': 128, 'strides': 1, 'padding': 1, 'activation_fct': 'ReLU', 'optimizer': 'Adam'}
block3_conv1: {'batchsize': 64, 'matsize': 56, 'kernelsize': 3, 'channels_in': 128, 'channels_out': 256, 'strides': 1, 'padding': 1, 'activation_fct': 'ReLU', 'optimizer': 'Adam'}
block3_conv2: {'batchsize': 64, 'matsize': 56, 'kernelsize': 3, 'channels_in': 256, 'channels_out': 25

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import time

# Check if GPU is available and set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transformations for the training data
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),  # Convert images to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])

# Create a random dataset (For demonstration, using FakeData)
train_dataset = datasets.FakeData(size=dataset_size, transform=transform, image_size=(3, 224, 224))  # 1000 samples
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Load VGG16 model
model = models.vgg16(pretrained=False)  # Set pretrained=True if you want to use ImageNet weights
model.classifier[6] = nn.Linear(model.classifier[6].in_features, 10)  # Change the final layer to 10 classes
model.to(device)


# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 100
start_time = time.time()
for epoch in tqdm.tqdm(range(epochs)):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss:.4f}')
end_time = time.time()
print("Training complete!")


/home/ubuntu/miniconda/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ubuntu/miniconda/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
  1%|          | 1/100 [00:08<14:19,  8.68s/it]

Epoch [1/100], Loss: 15.0384


  2%|▏         | 2/100 [00:18<15:07,  9.26s/it]

Epoch [2/100], Loss: 2.3020


  3%|▎         | 3/100 [00:27<15:08,  9.37s/it]

...

Epoch [98/100], Loss: 2.2963


 99%|█████████▉| 99/100 [14:16<00:08,  8.90s/it]

Epoch [99/100], Loss: 2.2977


100%|██████████| 100/100 [14:25<00:00,  8.65s/it]

Epoch [100/100], Loss: 2.2959
Training complete!


In [8]:
time_elapsed = end_time - start_time # Calculate the total time taken for training in seconds
print(f"Total time taken for training VGG16: {time_elapsed:.2f} seconds")

Total time taken for training VGG16: 865.44 seconds


In [9]:
time_for_one_epoch = time_elapsed / epochs
print(f"Average time taken for one epoch of VGG16: {time_for_one_epoch:.2f} seconds")

Average time taken for one epoch of VGG16: 8.65 seconds


In [10]:
gpu_specs = pd.read_csv('../gpus/GPUs.csv')
gpu_specs

,GPU,Provisioning,Base Clock (MHz),Boost Clock (MHz),Memory Clock (MHz),Memory (GB),Memory Type,Memory Bus (bit),GPU Memory Bandwidth (GB/s),Bus,...,TMUs,ROPs,SM,TC,RT,PR,TR,FP16,FP32,FP64
0,L4,Cloud,795,2040,1563,24,GDDR6,192,300,PCIe 4.0,...,240,80,60,240,60,163,490,30290,30290,473
1,P4,Cloud,886,1114,1502,8,GDDR5,256,192,PCIe 3.0,...,160,64,20,0,0,71,178,89,5700,178
2,P100,Cloud,1190,1329,715,16,HBM2,4096,732,PCIe 3.0,...,224,96,56,0,0,127,297,19050,9526,4763
3,RTX4090,Local,2235,2520,1313,24,GDDR6X,384,1001,PCIe 4.0,...,512,176,128,512,128,443,1290,82580,82580,1290
4,RTXA4000,Local,735,1560,1750,16,GDDR6,256,448,PCIe 4.0,...,192,96,48,192,48,150,300,19170,19170,300
5,T4,Cloud,585,1590,1250,16,GDDR6,256,320,PCIe 3.0,...,160,64,40,320,40,101,254,65130,8141,254
6,V100,Cloud,1245,1380,876,16,HBM2,4096,897,PCIe 3.0,...,320,128,80,640,0,176,441,28260,14130,7066


In [11]:
conv_layer_features = vgg16_conv_layers
dense_layer_features = vgg16_dense_layers

In [12]:
gpu = gpu_specs[gpu_specs['GPU'] == gpu_name]
gpu = gpu.squeeze()
gpu = pd.DataFrame([gpu] * len(conv_layer_features))

conv_layer_features = pd.DataFrame.from_dict(conv_layer_features, orient='index')
conv_layer_features = pd.concat([conv_layer_features.reset_index(drop=True), gpu.reset_index(drop=True)], axis=1)

gpu = gpu_specs[gpu_specs['GPU'] == gpu_name]
gpu = gpu.squeeze()
gpu = pd.DataFrame([gpu] * len(dense_layer_features))

dense_layer_features = pd.DataFrame.from_dict(dense_layer_features, orient='index')
dense_layer_features = pd.concat([dense_layer_features.reset_index(drop=True), gpu.reset_index(drop=True)], axis=1)

In [13]:
conv_layer_features['activation_fct'] = 1
conv_layer_features['optimizer'] = 4
conv_layer_features['precision'] = 32
conv_layer_features['padding'] = conv_layer_features['padding'].apply(lambda x: 'valid' if x == 0 else 'same')
conv_layer_features['use_bias'] = 0

In [14]:
dense_layer_features['activation_fct'] = 1
dense_layer_features['optimizer'] = 4
dense_layer_features['precision'] = 32

In [15]:
conv_layer_features

,batchsize,matsize,kernelsize,channels_in,channels_out,strides,padding,activation_fct,optimizer,GPU,...,SM,TC,RT,PR,TR,FP16,FP32,FP64,precision,use_bias
0,64,224,3,3,64,1,same,1,4,V100,...,80,640,0,176,441,28260,14130,7066,32,0
1,64,224,3,64,64,1,same,1,4,V100,...,80,640,0,176,441,28260,14130,7066,32,0
2,64,112,3,64,128,1,same,1,4,V100,...,80,640,0,176,441,28260,14130,7066,32,0
3,64,112,3,128,128,1,same,1,4,V100,...,80,640,0,176,441,28260,14130,7066,32,0
4,64,56,3,128,256,1,same,1,4,V100,...,80,640,0,176,441,28260,14130,7066,32,0
5,64,56,3,256,256,1,same,1,4,V100,...,80,640,0,176,441,28260,14130,7066,32,0
6,64,56,3,256,256,1,same,1,4,V100,...,80,640,0,176,441,28260,14130,7066,32,0
7,64,28,3,256,512,1,same,1,4,V100,...,80,640,0,176,441,28260,14130,7066,32,0
8,64,28,3,512,512,1,same,1,4,V100,...,80,640,0,176,441,28260,14130,7066,32,0
9,64,28,3,512,512,1,same,1,4,V100,...,80,640,0,176,441,28260,14130,7066,32,0


In [16]:
dense_layer_features

,batchsize,dim_input,dim_output,activation_fct,optimizer,GPU,Provisioning,Base Clock (MHz),Boost Clock (MHz),Memory Clock (MHz),...,ROPs,SM,TC,RT,PR,TR,FP16,FP32,FP64,precision
0,64,25088,4096,1,4,V100,Cloud,1245,1380,876,...,128,80,640,0,176,441,28260,14130,7066,32
1,64,4096,4096,1,4,V100,Cloud,1245,1380,876,...,128,80,640,0,176,441,28260,14130,7066,32
2,64,4096,10,1,4,V100,Cloud,1245,1380,876,...,128,80,640,0,176,441,28260,14130,7066,32


In [17]:
from predict.features import (
    PreprocessConvFeatures,
    PreprocessDenseFeatures,
)

conv_features = PreprocessConvFeatures(conv_layer_features, include_additional_features=True).features
dense_features = PreprocessDenseFeatures(dense_layer_features, include_additional_features=True).features

conv_features_less = PreprocessConvFeatures(conv_layer_features, include_additional_features=False).features
dense_features_less = PreprocessDenseFeatures(dense_layer_features, include_additional_features=False).features

/home/ubuntu/dl-training-time-prediction/predict/features.py:455: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['Adam' 'Adam' 'Adam' 'Adam' 'Adam' 'Adam' 'Adam' 'Adam' 'Adam' 'Adam'
 'Adam' 'Adam' 'Adam']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  self.features.loc[:, 'optimizer'] = self.features['optimizer'].map({0:'None',
/home/ubuntu/dl-training-time-prediction/predict/features.py:462: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['relu' 'relu' 'relu' 'relu' 'relu' 'relu' 'relu' 'relu' 'relu' 'relu'
 'relu' 'relu' 'relu']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  self.features.loc[:, 'activation_fct'] = self.features['activation_fct'].map({0:'None',
/home/ubuntu/dl-training-time-prediction/predict/features.py:472: SettingWithCopyWarning: 
A value

In [18]:
conv_features

,Base Clock (MHz),Boost Clock (MHz),Bus_PCIe 3.0,Bus_PCIe 4.0,Cores,FP32,GPU,GPU Memory Bandwidth (GB/s),Memory (GB),Memory Bus (bit),...,optimizer_Adagrad,optimizer_Adam,optimizer_None,optimizer_RMSProp,optimizer_SGD,padding_same,padding_valid,precision,strides,use_bias
0,1245,1380,1.0,0.0,5120,14130,V100,897,16,4096,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,32,1,0
1,1245,1380,1.0,0.0,5120,14130,V100,897,16,4096,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,32,1,0
2,1245,1380,1.0,0.0,5120,14130,V100,897,16,4096,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,32,1,0
3,1245,1380,1.0,0.0,5120,14130,V100,897,16,4096,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,32,1,0
4,1245,1380,1.0,0.0,5120,14130,V100,897,16,4096,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,32,1,0
5,1245,1380,1.0,0.0,5120,14130,V100,897,16,4096,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,32,1,0
6,1245,1380,1.0,0.0,5120,14130,V100,897,16,4096,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,32,1,0
7,1245,1380,1.0,0.0,5120,14130,V100,897,16,4096,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,32,1,0
8,1245,1380,1.0,0.0,5120,14130,V100,897,16,4096,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,32,1,0
9,1245,1380,1.0,0.0,5120,14130,V100,897,16,4096,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,32,1,0


In [19]:
dense_features

,Base Clock (MHz),Boost Clock (MHz),Bus_PCIe 3.0,Bus_PCIe 4.0,Cores,FP32,GPU,GPU Memory Bandwidth (GB/s),Memory (GB),Memory Bus (bit),...,memory_out,memory_total,memory_weights,optimizer_Adadelta,optimizer_Adagrad,optimizer_Adam,optimizer_None,optimizer_RMSProp,optimizer_SGD,precision
0,1245,1380,1.0,0.0,5120,14130,V100,897,16,4096,...,262144,209256448,102760448,0.0,0.0,1.0,0.0,0.0,0.0,32
1,1245,1380,1.0,0.0,5120,14130,V100,897,16,4096,...,262144,34603008,16777216,0.0,0.0,1.0,0.0,0.0,0.0,32
2,1245,1380,1.0,0.0,5120,14130,V100,897,16,4096,...,640,607488,40960,0.0,0.0,1.0,0.0,0.0,0.0,32


In [20]:
conv_features = conv_features.drop(columns=['GPU'])
dense_features = dense_features.drop(columns=['GPU'])

conv_features_less = conv_features_less.drop(columns=['GPU'])
dense_features_less = dense_features_less.drop(columns=['GPU'])

In [21]:
predicted_conv_less = conv_wo_features_predictor.predict(conv_features_less)
predicted_dense_less = dense_wo_features_predictor.predict(dense_features_less)

predicted_conv_less = sum(predicted_conv_less * (dataset_size / batch_size)) / 1000
predicted_dense_less = sum(predicted_dense_less * (dataset_size / batch_size)) / 1000

In [22]:
predicted_conv = conv_predictor.predict(conv_features)
predicted_dense = dense_predictor.predict(dense_features)

predicted_conv = sum(predicted_conv * (dataset_size / batch_size)) / 1000
predicted_dense = sum(predicted_dense * (dataset_size / batch_size)) / 1000

In [23]:
predicted_e2e = predicted_conv + predicted_dense
predicted_e2e_less = predicted_conv_less + predicted_dense_less

In [24]:
print(f"Error in prediction with all features for VGG16: {abs(predicted_e2e - time_for_one_epoch) / time_for_one_epoch * 100:.2f}%")
print(f'Predicted time for one epoch will all features for VGG16: {predicted_e2e:.2f} seconds')
print(f'Actual time for one epoch: {time_for_one_epoch:.2f} seconds')

Error in prediction with all features for VGG16: 13.96%
Predicted time for one epoch will all features for VGG16: 7.45 seconds
Actual time for one epoch: 8.65 seconds


In [25]:
print(f"Error in prediction with raw features for VGG16: {abs(predicted_e2e_less - time_for_one_epoch) / time_for_one_epoch * 100:.2f}%")
print(f'Predicted time for one epoch with raw features for VGG16: {predicted_e2e_less:.2f} seconds')
print(f'Actual time for one epoch: {time_for_one_epoch:.2f} seconds')

Error in prediction with raw features for VGG16: 78.87%
Predicted time for one epoch with raw features for VGG16: 1.83 seconds
Actual time for one epoch: 8.65 seconds
